In [96]:
# %load src/header.py
'''LOAD'''
# %load /Users/rokushou/Desktop/header.py
'''GENERAL'''
import os
#import time
#import timeit
#import warnings
#warnings.simplefilter('ignore')
import random
import collections
#import secrets
import math
import itertools
'''DATA'''
import numpy as np
import pandas as pd
# import databricks.koalas as ks
'''MONGO'''
#from pymongo import MongoClient
'''PLOT'''
import matplotlib.pyplot as plt
plt.style.use('seaborn-pastel')
font = {'size':16}
import seaborn as sns
'''SCIPY/STATS'''
#import scipy.stats as scs
# from scipy.optimize import nnls
#import statsmodels.api as sm
'''SCIKIT LEARN'''
#from sklearn.datasets import make_classification, load_iris, load_boston, load_digits
#from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, KFold, ShuffleSplit, cross_val_score, GridSearchCV, cross_val_predict
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, r2_score, mean_squared_error, classification_report, make_scorer
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
#from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor, GradientBoostingClassifier, AdaBoostRegressor, AdaBoostClassifier
# from sklearn.decomposition import PCA, NMF
# from sklearn.cluster import KMeans
#from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
'''TENSORFLOW'''
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.preprocessing import text
from tensorflow.keras import utils
'''NLTK'''
#import nltk
#import unicodedata
#import string
#from nltk.corpus import stopwords
#from nltk.tokenize import sent_tokenize, word_tokenize
#from nltk.stem.porter import PorterStemmer
#from nltk.stem.snowball import SnowballStemmer
#from nltk.stem.wordnet import WordNetLemmatizer
'''NETWORK'''
# import networkx as nx
# import nxpd
# import community as comm
'''FUNCTIONS'''
nrange = lambda x : itertools.repeat(None, x)
def argpsort(a, k):
    return np.argpartition(a, range(k))[:k]
def arghsort(a, k):
    b = np.argpartition(a, k)[:k]
    return b[np.argsort(a[b])]
'''PALETTE'''
t = 'setsuna'
qan = {'setsuna':'quanta'}
twi = '#DCB8E7' #Pale, light grayish mulberry
twi_blu = '#273873' #Dark sapphire blue
twi_pur = '#662D8A' #Moderate purple
twi_pnk = '#ED438D' #Brilliant raspberry
'''RETURN'''
!fortune | cowsay -f dragon


 _________________________________________ 
/ K: Cobalt's metal, hard and shining;    \
|                                         |
| Cobol's wordy and confining;            |
|                                         |
| KOBOLDS topple when you strike them;    |
|                                         |
| Don't feel bad, it's hard to like them. |
|                                         |
\ -- The Roguelet's ABC                   /
 ----------------------------------------- 
      \                    / \  //\
       \    |\___/|      /   \//  \\
            /0  0  \__  /    //  | \ \    
           /     /  \/_/    //   |  \  \  
           @_^_@'/   \/_   //    |   \   \ 
           //_^_/     \/_ //     |    \    \
        ( //) |        \///      |     \     \
      ( / /) _|_ /   )  //       |      \     _\
    ( // /) '/,_ _ _/  ( ; -.    |    _ _\.-~        .-~~~^-.
  (( / / )) ,-{        _      `-.|.-~-.           .~         `.
 (( // / ))  '/\      /                 ~-. _ 

# EDA
First I load in my cleaned data.

In [2]:
df = pd.read_pickle('data/clean.pkl')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12769497 entries, 0 to 15623
Data columns (total 10 columns):
match         int64
game-ver      object
lobby-mode    object
lobby         object
mode          object
stage         object
weapon        object
rank          object
level         float64
win           bool
dtypes: bool(1), float64(1), int64(1), object(7)
memory usage: 986.4+ MB


It is much more data than I need and very slow to work with so I will select only the most recent (based on game version) and work with that subset.

In [3]:
df['game-ver'].unique()

array(['4.2.0', '4.3.0', '4.3.1', '3.0.0', '1.3.0', '3.0.1', '4.4.0',
       '4.5.0', '4.5.1', '4.6.0', '4.6.1', '4.7.0', '2.2.2', '1.4.2',
       '2.0.0', '3.1.0', '3.2.0', '4.8.0', '3.2.1', '3.2.2', '1.1.2',
       '4.0.0', '4.1.0', '2.3.3', '2.3.2', '1.4.0', '2.2.1', '2.1.0',
       '2.1.1', '2.3.1', '1.4.1', '1.2.0', '2.2.0', '2.0.1', '2.3.0',
       '1.0.0'], dtype=object)

In [4]:
df = df[df['game-ver']=='4.8.0']
df.shape

(396655, 10)

40k is still plenty and I may decide to random sample from it if my models are stil taking too long. The features I am most intersted in are mode, weapon, and rank.

In [5]:
df['mode'].unique()

array(['yagura', 'asari', 'area', 'nawabari', 'hoko'], dtype=object)

In [6]:
df['rank'].unique()

array(['s', 'a', 'a+', 'a-', nan, 'x', 's+', 'c-', 'b-', 'b', 'c+', 'b+',
       'c'], dtype=object)

In [7]:
df['weapon'].unique()

array(['ochiba', 'dynamo_tesla', 'sshooter_becchu', 'wakaba',
       'jetsweeper', 'clashblaster_neo', 'prime_becchu',
       'maneuver_collabo', 'jetsweeper_custom', 'nzap89',
       'sputtery_clear', 'sharp_neo', 'dualsweeper_custom', 'bold_7',
       'momiji', 'carbon_deco', 'splatroller', 'spygadget_becchu',
       'hokusai_hue', 'maneuver', '52gal', 'hotblaster', 'furo',
       'sputtery_hue', 'bold', 'variableroller_foil', 'longblaster',
       'pablo', 'sshooter_collabo', 'hydra_custom', 'nzap85',
       'screwslosher', 'explosher', 'l3reelgun_d', 'pablo_hue',
       'prime_collabo', 'promodeler_pg', 'spygadget', 'nzap83',
       'sputtery', 'kelvin525_deco', 'liter4k_scope_custom',
       'hokusai_becchu', 'kugelschreiber_hue', 'screwslosher_neo',
       'maneuver_becchu', 'splatscope', 'promodeler_mg', 'bold_neo',
       'barrelspinner_remix', 'herospinner_replica', 'nautilus47',
       'h3reelgun_d', 'dualsweeper', 'liter4k_custom',
       'hotblaster_custom', 'nova', 'bamboo

Break the dataframe into features and labels, then get dummies for the categorical features.

In [98]:
X = df[['lobby', 'mode', 'stage', 'weapon', 'rank', 'level']]
y = df.win

In [99]:
X = pd.get_dummies(X)
X.shape

(396655, 186)

In [100]:
# scaler = StandardScaler()
# scaler.fit_transform(X)

That created a lot of features but I have enough observations. Next to create a basic test train split and model.

In [101]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

### Logistic Regression

In [16]:
lr = LogisticRegression(solver='lbfgs')
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.517399176451714

### Random Forest

In [19]:
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.5197903695271855

### Gradient Boosting

In [18]:
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)
gb.score(X_test, y_test)

0.5175443287470302

### SGD

In [112]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
sgd.score(X_test, y_test)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.5110277546467833

### Neural Networks

In [130]:
X = pd.get_dummies(df[['lobby', 'mode', 'stage', 'weapon', 'rank', 'level']])
y = df.win
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [131]:
model = Sequential()
model.add(Dense(units=50, input_shape=(X_train.shape[1],)))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=4000, verbose=1, validation_split=0.1)

Train on 239182 samples, validate on 26576 samples
Epoch 1/10
239182/239182 [==============================] - 3s 13us/sample - loss: 0.7378 - accuracy: 0.5005 - val_loss: 0.6959 - val_accuracy: 0.5035
Epoch 2/10
239182/239182 [==============================] - 2s 10us/sample - loss: 0.6944 - accuracy: 0.5095 - val_loss: 0.6931 - val_accuracy: 0.5058
Epoch 3/10
239182/239182 [==============================] - 3s 11us/sample - loss: 0.6920 - accuracy: 0.5186 - val_loss: 0.6926 - val_accuracy: 0.5062
Epoch 4/10
239182/239182 [==============================] - 3s 11us/sample - loss: 0.6915 - accuracy: 0.5213 - val_loss: 0.6928 - val_accuracy: 0.5062
Epoch 5/10
239182/239182 [==============================] - 3s 11us/sample - loss: 0.6918 - accuracy: 0.5198 - val_loss: 0.6937 - val_accuracy: 0.5160
Epoch 6/10
239182/239182 [==============================] - 2s 10us/sample - loss: 0.6913 - accuracy: 0.5219 - val_loss: 0.6933 - val_accuracy: 0.5166
Epoch 7/10
239182/239182 [=================

In [132]:
confusion_matrix(y_test, model.predict_classes(X_test))

array([[35997, 30097],
       [32923, 31880]])